In [113]:
import os
import json
from difflib import SequenceMatcher
def search_relevance (score):
        match,relevance = "no",""
        if score>=99:
            match,relevance = "full",""
        elif score >=90:
            match,relevance = "partial","high"
        elif score >=70:
            match,relevance = "partial","medium"
        elif 0<score <70:
            match,relevance = "partial","low"
        return match,relevance

In [114]:
import json
import re
import fitz

from pdf2txt import convert_pdf
from bs4 import BeautifulSoup,NavigableString
from difflib import SequenceMatcher
import os


from nltk.corpus import stopwords

In [115]:
def getMatchTypeRelevance(a) : 

    matchtype_agg=""
    relevance_agg=""
    for i in a:
        matchtype = i.get("matchType","")
        relevance = i.get("relevance","")
        
        
        if matchtype=="partial" :
            
            if matchtype_agg not in("no"):
                matchtype_agg=matchtype
            elif matchtype_agg=="":
                matchtype_agg=matchtype
                
            
                
        elif matchtype=="full" :
            if matchtype_agg not in("partial","no"):
                matchtype_agg=matchtype
            elif matchtype_agg=="":
                matchtype_agg=matchtype
        elif matchtype=="no":
            matchtype_agg=matchtype
        
        if relevance=="low" :
            relevance_agg=relevance
        elif relevance=="medium" :
            
            if relevance_agg not in("low"):
                relevance_agg=relevance
            elif relevance_agg=="":
                relevance_agg=relevance
            
        elif relevance=="high" :
            
            if relevance_agg not in("medium","low"):
                relevance_agg=relevance
            elif relevance_agg=="":
                relevance_agg=relevance
            
    if matchtype_agg=="no" or matchtype_agg=="full":
        relevance_agg=""
   
    return matchtype_agg,relevance_agg

In [116]:
from tabula import read_pdf

In [117]:
import fitz

In [49]:
def get_html(doc_name,pages,doc_path=r"C:\Users\LATEJAS\Videos\PDBI"):
    paragraph=""
    doc=fitz.open(os.path.join(doc_path,doc_name))
    for page_no in pages:

        page = doc.loadPage(page_no-1)
        paragraph=paragraph+page.getText("html")+"\n"
    return paragraph

In [50]:
def generate_clean_html(doc_name,pages,doc_path=r"C:\Users\LATEJAS\Videos\PDBI"):
    fitz_doc = fitz.open(os.path.join(doc_path,doc_name))
    html_full=''
    wording_ref=[]
    footer_dict={}
    total_pages=len(fitz_doc)
#     print(total_pages)
    for page_no in pages:
        page = fitz_doc.loadPage(page_no-1)
        

        page_html=page.getText("html")
       
        page_html=page_html.replace('position','page:{};position'.format(page_no))
        soup = BeautifulSoup(page_html, 'html5lib')
        toc_page_no=[]
        dot_count=0
        introflag=False

        for p in soup.findAll('p'):        
            p_str=str(p)
            dot_count+=p_str.count('.')

            try:        
                xcord=int(x_cordinate_Re.findall(p_str)[0])
            except:
                xcord=0 
            try:
                ycord=int(y_cordinate_Re.findall(p_str)[0])
            except:
                ycord=0

            for span in p.findAll('span'):
                text=span.text.strip()
                try:
                    toc_page=int(span.text)


                    if page_no< total_pages and xcord>500:
                        toc_page_no.append(toc_page)                   
                except Exception as err:
                    pass
                try:
                    font_size=float(font_size_RE.search(p_str).group(1))
                except Exception as err:
                    print(err)
                    font_size=8

                if xcord < 150 and ycord >720 and font_size <10 :


                    wording_ref.append(text)
                if  ycord >720  and font_size <10 :
                    footer_dict[text]=ycord

 

 

 


        if len(toc_page_no) >10 or dot_count>1500 :
#             print('table of content',page_no)
            continue
        if 'img' in page_html:
#             print('Image',page_no)
            continue
        if 'Introduction' in page_html and 'www.chubb.com' in page_html:
#             print('Intro',page_no)
            continue

#         if page_no <=16:
#             df=read_pdf(file,  pages=page_no)
#             if df:
#                 print('TABLE FOUND IN ',page_no)
#                 continue


        html_full+=page_html

    wording_ref=[i for i in wording_ref if len(i)>2]
    wording_reference_p_text=max(set(wording_ref), key = wording_ref.count)
    footer_y_cord=footer_dict[wording_reference_p_text]-20
    wording_reference=[i for i in wording_reference_p_text.split('    ') if len(i)>0][0]
    
    html_full=html_full.replace('width:-2pt;height:-2pt','width:596pt;height:842pt')

 

 

 

    soup = BeautifulSoup(html_full, 'html5lib')

    for p in soup.findAll('p'):
        p_str=str(p)
        text=p.text.strip()


        try:        
            xcord=int(x_cordinate_Re.findall(p_str)[0])
        except:
            xcord=0 
        try:
            ycord=int(y_cordinate_Re.findall(p_str)[0])
        except:
            ycord=0

        try:
            font_size=float(font_size_RE.search(p_str).group(1))
        except Exception as err:
            print(err)
            font_size=8
        if "policy number:" in p.text.lower() and ycord <100:
#             print(p.text)
            p.string=''
        if  font_size >40 :
#             print(p.text,'WATERMARKKK')
            p.string=''
    
        if len(p.text)>4:    
            if ycord >footer_y_cord and (font_size <8 or (p.text.strip()==wording_reference.strip())):
#                 print(p.text,'refff')
                p.string=''
        if font_size < 7 :
#             print(p.text,'less than 7 ')
            p.string=''
    

 

 

    return soup,wording_reference

In [8]:
# html_content = str(soup1)
# Html_file= open("doc_mod.html","w",encoding='utf-8')
# Html_file.write(html_content)
# Html_file.close()


In [51]:

pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
stop_words = list(set(stopwords.words('english')))
y_cordinate_Re=re.compile(';top:(\d{1,5})pt')  #top:766pt
x_cordinate_Re=re.compile(';left:(\d{1,5})pt')
height_re=re.compile('height:(\d{1,5})pt')
font_size_RE = re.compile("font-size:(.*?)pt")
 


In [52]:
import diff_match_patch as dmp_module

dmp = dmp_module.diff_match_patch()


def get_dif(str1,str2):
    dmp.Diff_Timeout = 0
    
#     if 10000 <len(str2)<20000:
#          dmp.Diff_Timeout = 20
#     elif 20000<=len(str2)<25000:
#         dmp.Diff_Timeout = 60
#     elif 25000<=len(str2)<30000:
#         dmp.Diff_Timeout = 120
#     elif 30000<=len(str2)<35000:
#         dmp.Diff_Timeout = 180
#     elif len(str2)>35000:
#         dmp.Diff_Timeout = 240
    

#     print(        dmp.Diff_Timeout )   
    d = dmp.diff_main(str1, str2)
    dmp.diff_cleanupSemantic(d)

    paragraph=dmp.diff_prettyHtml( d)
    return paragraph,d

In [53]:
def sub_match(str1,str2):

    if str1 in str2 or str2 in str1:
        return True
    return False

In [598]:
def long_substring(string1,string2):
    
    if string1 in string2:
#         print(string1)
        return True,string1
    sequence_matcher = SequenceMatcher(
        None, string1, string2)
    match = sequence_matcher.find_longest_match(
        0, len(string1), 0, len(string2))
    answer=string1[match.a:match.a+match.size]
#     if len(answer.strip())<4:
#         return False,""
#     print(answer)
    return True ,answer

In [589]:
# %%time
# import os
# word_exclusion=word_content[0]["exclusion"]
# doc_exclusion=doc_content[0]["exclusion"]

 

In [699]:



from azure.cosmos import exceptions, CosmosClient, PartitionKey

cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
client = CosmosClient(cosmo_endpoint, cosmo_key)
database=client.get_database_client('policy-analysis')
tt_documents_container=database.get_container_client('trace_files_dev')
trace_result_container=database.get_container_client('trace_results_dev')

# In[12]:


# wording_query="SELECT * FROM c WHERE c.file_type ='base wording' "
wording_query="SELECT * FROM c WHERE c.title ='Chubb Multinational PDBI - UK - 04-2018 (SE - from 01-01-2019) - SPECIMEN'"

wordings = list(tt_documents_container.query_items(
    query=wording_query,
    enable_cross_partition_query=True
))


# In[19]:


# document_query="SELECT * FROM c WHERE c.file_type ='policy document'  "
document_query="SELECT * FROM c WHERE c.title ='REDACTED_3284C99B-3F17-4BE8-855B-6F44AF08541F'"

documents = list(tt_documents_container.query_items(
    query=document_query,
    enable_cross_partition_query=True
)) 



result_query="SELECT * FROM c"



results = list(trace_result_container.query_items(
    query=result_query,
    enable_cross_partition_query=True
))


In [700]:
wording_content=wordings[0]['contents']
document_content=documents[0]['contents']


document_name=list(document_content.keys())[0]
# print("document_name",document_name)
wording_name=list(wording_content.keys())[0]
doc_content=document_content[document_name]
word_content=wording_content[wording_name]

In [701]:
def get_html_with_diff(doc_name,pages,key_values,dist_,start,edgevalues):
    soup,wording_reference=generate_clean_html(doc_name,pages)
    soup_p_list=[]

    soup= get_highlights(key_values,soup,start)
    

    total=0
    for i,item in enumerate(dist_):
        
        
        p_index=-total

        
        tag,s_text=item[1].strip().split("$")
        total=total+13+len(item[0])
        searchtext=" ".join(s_text.split()[:4])

        checkFalg=False 
        total_text=""
        for p in soup.findAll('p'):
            text=p.text

            total_text+=text
            
            dist_count=text.count("<dist>")
            original_text=text
            text=text.replace("<ext>","").replace("</ext>","")
            

            if p_index<=int(tag)<=p_index+len(text)  and len(s_text.strip())<3:
                p.string ="<dist>{}</dist>".format(item[0])
                break
            if p_index<=int(tag)<=p_index+len(text) and len(text.strip())>1 :
                
                
                flag,answer=long_substring(searchtext,text)
               
                if flag:
                    p.string=add_dist_tag(original_text,answer,item[0])
                    print("-----------------------------------------------------------------")

                    break
            if start in text: 
                checkFalg=True
            if checkFalg:
                p_index+=len(text)
            
     

                
                
    paragraph=str(soup)
    if len(edgevalues)>=2:
        print(edgevalues[0][1])
        find_string=edgevalues[0][1].split()[-1]
        value = paragraph.rfind(find_string)
        if value!=-1:
            value+=len(find_string)
            paragraph= paragraph[:value] + "&lt;dist&gt;{}&lt;/dist&gt;".format(edgevalues[1][1]) + paragraph[value + 1:]
            
    paragraph=paragraph.replace("color:#ffffff","")
    return paragraph

In [702]:
def get_highlights(key_values,soup,start) :
    i= start
#     print(i)
    highlight_index={}
    total=0
    for index1,key in key_values.items():
        content =""
        total_len=0

        flag=False

        key_count=0
        for  p in  soup.find_all('p'):
            a=p.text

            replace_text=p.text.replace("<ext>","").replace("</ext>","")



            checkFlag,answer=    long_substring(key.strip(),replace_text)
            
            if len(a.strip())>6 and checkFlag :


                try:


                    p_text_len=total_len+a.index(key,key_count)


                    if index1==p_text_len:
                        new_tag = soup.new_tag("span")
                        p.string=a.replace(key,"<ext>{}</ext>".format(key))
                        key_count+=len(key)


                except:



                    try:

                        if index1+key.index(a,key_count)-40<=total_len<=index1+key.index(a,key_count) :
                            p.string=a.replace(a,"<ext>{}</ext>".format(a))
                            key_count+=len(a)
                            print(p.string)






                    except:

                        if total_len+replace_text.index(answer)==index1+key.index(answer) and answer is not None:

                            p.string=a.replace(answer,"<ext>{}</ext>".format(answer))





            if a.strip() and  a.startswith(start) and not flag :

                flag=True
            if flag :

                total_len+=len(a.replace("<ext>","").replace("</ext>",""))

                content+=a

#         print("p",content)
    return soup

In [703]:
import re

def add_dist_tag(b,a,add):


    tag_re=re.compile(r'</*ext>|</*dist>')

 


    match_d={}
    matches = tag_re.finditer(b)
    for match in matches:
        match_d[match.start()]=(match.group(),len(match.group()))



    clean_b=tag_re.sub('',b)
    replace_string="<dist>{}</dist>".format(add)
    new_added=clean_b.replace(a,replace_string+a)
    replace_pos=new_added.find(replace_string)
    prev_tag_len=0
    for key,value in match_d.items():
        if key >=replace_pos+prev_tag_len:
            key=key+len(replace_string)   



        new_added=new_added[:key] + value[0] + new_added[key:]
        prev_tag_len+=len(value[0])
    return new_added

In [704]:
def get_differences(str1,str2,category):


    diff_,get_=get_dif(str1,str2)
   
    f=open(category+"pretty.html","w",encoding="utf-8")
    f.write(diff_)

    removedminus=[ i for i in get_ if i[0]!=-1]

    
    start=""
    
        
    key_values={}
    total =0
    for index,a in enumerate(removedminus):
        item,value=a

        if item==1:
            

            key_values[total]=value
       
        total+=len(value)
    
    
    pre_text=""
    pre_tag=0
    dist_=[]
    search_highlight={}
    flag=True
    total=0
    total_text=""
    
    for item in get_:
        
        tag,text=item
        total_text+=text
        if len(text.strip())>3 and (tag==0 or tag==1) and flag :
            flag=False
            start = text.split()[0]
#             print(start)
            if start.lower() in stop_words:
                start=text[:50]
        
        if tag==0:
            prev_index=tag
            prev_value= text
        if pre_tag==-1:
            extra_spaces=len(value) - len(value.lstrip(' '))
            
            total1=extra_spaces+total           
            dist_.append((pre_text,str(total1)+"$"+text))

            
        pre_tag=tag
        pre_text=text

        if tag==0 or tag==1:
            total+=len(text)
        
    dist_=[dist for dist in dist_ if len(dist[1].split('$')[1].strip()) >=1 and len(dist[0].strip()) >3 ]
    key_values={key:value for key,value in key_values.items() if len(value.strip())>=3 and refCode_document!=value}
    get_= [(i[0],i[1]) for i in get_ if len(i[1].strip())>1]
    edgevalues=""
    if get_[-1][0]==-1:
        edgevalues=get_[-2:]
    print("edgevalues",edgevalues)
    return key_values,dist_,start,edgevalues
    

In [705]:
word_exclusion=word_content[0]["exclusion"]
doc_exclusion=doc_content[0]["exclusion"]
key="Section 1 Property Damage and Section 2 Business Interruption"

f= open("final2"+".html","w",encoding="utf-8")
pages=doc_exclusion[key][1]
pages.sort()

key_values,dist_,start,edgevalues=get_differences(word_exclusion[key][0],doc_exclusion[key][0],'exclusion')
# start="Data"
# print("start",start)
# start="Condition#"
# print("$$$$$$$$$$$$$$$$$$$$$$$$$$")
# start="Exclusion"


soup= get_html_with_diff(document_name,pages,key_values,dist_,start,edgevalues)
paragraph=str(soup)
paragraph = paragraph.replace("&lt;ext&gt;","<mark>")
paragraph = paragraph.replace("&lt;/ext&gt;","</mark>")



f.write(paragraph)

edgevalues 
<ext>Business Interruption: </ext>
<ext>Property Damage: </ext>
<ext>The Company shall not be liable under this Extension in respect of damage to equipment: </ext>
<ext>in respect of which there is no guarantee, maintenance, rental, hire or lease agreement in force </ext>
<ext>providing a minimum service of on-call remedial or corrective maintenance at inclusive cost, other </ext>
<ext>than damage occasioned by or happening through: </ext>
<ext>the negligence of the Insured, or </ext>
<ext>the failure or fluctuation of the supply of electricity to the equipment; </ext>
<ext>caused by or occasioned through: </ext>
<ext>the intentional act or wilful neglect of the Insured; </ext>
<ext>wear and tear, gradual deterioration, gradually developing defects, corrosion, rust or </ext>
<ext>any testing, repairing, adjusting, servicing or maintenance operation. </ext>
<ext>Business Interruption: </ext>
<ext>The Company shall not be liable under this Extension in respect of:- </ext>
<ex

92731

In [706]:
dist_

[(' yards, car-parks, roads, pavements, walls, gates or fences',
  '2724$ unless '),
 (' land, roads, pavements,',
  '6799$ piers, jetties, bridges, culverts or excavations; g antiques, works of art, rare books, jewellery, precious stones, articles of gold, silver or other precious metals or furs; h property covered by any marine policy of insurance; i securities and other negotiable instruments, coin, bank and currency notes, cheques, National Giro payment orders, money and postal orders, postage and revenue stamps, National Savings stamps and certificates, holiday pay stamps, trading stamps, luncheon vouchers, credit cards and credit card vouchers;'),
 ('Money', '11355$Computer Breakdown'),
 (': This Extension does not cover:- a loss or destruction of or damage to money contained in coin operated gaming or amusement devices or vending machines; b loss or destruction of or damage to Money contained in any vehicle while left unattended c any loss, destruction or damage:- i. arising out

'Section 1 Property Damage '

In [125]:



# from azure.cosmos import exceptions, CosmosClient, PartitionKey

# cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
# cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
# client = CosmosClient(cosmo_endpoint, cosmo_key)
# database=client.get_database_client('policy-analysis')
# tt_documents_container=database.get_container_client('trace_files_dev')
# trace_result_container=database.get_container_client('trace_results_dev')

# # In[12]:


# wording_query="SELECT * FROM c WHERE c.file_type ='base wording' "

# wordings = list(tt_documents_container.query_items(
#     query=wording_query,
#     enable_cross_partition_query=True
# ))


# # In[19]:


# document_query="SELECT * FROM c WHERE c.file_type ='policy document'  "

# documents = list(tt_documents_container.query_items(
#     query=document_query,
#     enable_cross_partition_query=True
# ))


# # In[29]:
# result_query="SELECT * FROM c"



# results = list(trace_result_container.query_items(
#     query=result_query,
#     enable_cross_partition_query=True
# ))



        




    



In [ ]:
results_id=[i['id'] for i in results]

In [ ]:
len(results_id)

In [ ]:
k=0
for wording in wordings:
    
    refCode_wording=wording["refCode"]

    for document in documents:
        refCode_document=document["refCode"]
        wording_start_date=wording['effectiveDate'] 
        wording_end_date=wording['expiryDate'] 
        document_start_date=document['effectiveDate']
        if refCode_document==refCode_wording and  wording_start_date<=document_start_date<=wording_end_date:
            k+=1
            print(k)

In [ ]:
len(documents)

In [ ]:
document_set=documents[50:90]

In [62]:


result_query="SELECT * FROM c"

results = list(trace_result_container.query_items(
    query=result_query,
    enable_cross_partition_query=True
))

results_id=[i['id'] for i in results]

final_list=[]
for wording in wordings:
    
    refCode_wording=wording["refCode"]

    for document in documents:
        refCode_document=document["refCode"]
        wording_start_date=wording['effectiveDate'] 
        wording_end_date=wording['expiryDate'] 
        document_start_date=document['effectiveDate']
        try:
            if refCode_document==refCode_wording:            
                wording_content=wording['contents']
                document_content=document['contents']
                


                document_name=list(document_content.keys())[0]

                wording_name=list(wording_content.keys())[0]
                id_name=wording_name.replace(".pdf","")+document_name.replace(".pdf","")
                print(id_name)
                
                
                if id_name in results_id:
                    print('Already Exist........')
                    continue

                final_list.append([wording_name,document_name,refCode_document,refCode_wording,wording["effectiveDate"],document["effectiveDate"]])

                doc_content=document_content[document_name]
                word_content=wording_content[wording_name]

                print(document_name)
                matches={}

                total_category_MR=[]
                word_content[0].pop("pages", None)
                doc_content[0].pop("pages", None)

                for category in word_content[0].keys():

                    category_content_wording=word_content[0][category]
                    category_content_document=doc_content[0][category]
                    results=[]
                    total_section_MR=[]
                    total_section_score=0
                    total_sections=0
    #                 print(category_content_document.keys())




                    for section_w in category_content_wording.keys():
                        new_dict ={}
                        keyFlag=False
                        ##keys are present in both the wording and document

                        if section_w not in category_content_document.keys():
                            for section_d in category_content_document.keys():

                                if section_w in section_d or section_d in section_w:
                                    key_wording=section_w
                                    key_document=section_d
                                    keyFlag=True


                        else:
                            key_wording=section_w
                            key_document=section_w

                            keyFlag=True

                        if keyFlag:
    #                         print(key_wording,"*",key_document,category)
                            section_content,pages=category_content_document[key_document]

                            wording_section_value,wording_section_pages=category_content_wording[key_wording]
                            pages.sort()

                            wording_section_pages.sort()
                            ##when wording pages and document pages are there we will compare
                            if len(pages)>=1 and len(wording_section_pages)>=1:

                                key_values,dist_,start=get_differences(wording_section_value,section_content,category)
                                

                                score=SequenceMatcher(None,wording_section_value,section_content).ratio()*100
                                if not key_values and not dist_:
                                    score=100
                                if "SPECIAL EXTENSIONS APPLICABLE" in  section_w :
                                    score=100
                                    

                                matchType,relevance=search_relevance(score)
                                total_section_score+=score
                                total_sections+=1
                                new_dict["section"]=key_document


                                if (key_values or  dist_) and score <99:


                                    content_html= get_html_with_diff(document_name,pages,key_values,dist_,start)


                                    new_dict["content"]=content_html

                                new_dict["relevance"] = relevance
                                new_dict["matchType"] = matchType
                                new_dict["score"]=score
                                new_dict["document_page"]=pages[0]


                                new_dict["wording_page"]=wording_section_pages[0]


                                results.append(new_dict)
                                total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})

                            else:
                                ##if there are documnet pages
                                ##elif there are wording pages
                                if len(pages)>=1:
                                    new_dict["section"]=key_document
                                    new_dict["content"]="<ext>{}</ext>".format(get_html(document_name,pages))

                                    new_dict["document_page"]=pages[0]
                                    new_dict["relevance"] = ""
                                    new_dict["matchType"] = "no"
                                    new_dict["score"]=0

                                    results.append(new_dict)
                                    total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})


                                elif len(wording_section_pages)>=1:
                                    new_dict["section"]=key_wording
                                    new_dict["content"]="<dist>{}</dist>".format(get_html(wording_name,wording_section_pages,doc_path=r"C:\Users\LATEJAS\Videos\PDBI\wordings"))

                                    new_dict["wording_page"]=wording_section_pages[0]
                                    new_dict["relevance"] = ""
                                    new_dict["matchType"] = "no"
                                    new_dict["score"]=0

                                    results.append(new_dict)
                                    total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})




                        ###if the key it not there in document we should show wording pages with "No match"
                        if not keyFlag:
                            
                            _,pages_word=category_content_wording[section_w]

                            if len(pages_word)>=1:
                                pages_word.sort()
                                new_dict["section"]=section_w


                                new_dict["content"]="<dist>{}</dist>".format(get_html(wording_name,pages_word,doc_path=r"C:\Users\LATEJAS\Videos\PDBI\wordings"))
                                new_dict["relevance"] = ""
                                new_dict["matchType"] = "no"
                                new_dict["score"]=0
                                new_dict["wording_page"]=pages_word[0]
                                results.append(new_dict)
                                total_section_MR.append({"relevance":new_dict["relevance"],
                                                                 "matchType":new_dict["matchType"]})
                    total_section_matchType,total_section_relevance=getMatchTypeRelevance(total_section_MR)
                    matches[category]= {"results":results,
                                        "matchType": total_section_matchType,"relevance":total_section_relevance}
                    total_category_MR.append({"relevance":total_section_relevance,
                                                         "matchType":total_section_matchType})  

                total_category_matchType,total_category_relevance=getMatchTypeRelevance(total_category_MR)
                
                insert_item={
                              "policyDetails": {
                                    "policyNo": document["policyNo"],
                                    "expiryDate": document[ "expiryDate"],
                                    "effectiveDate": document["effectiveDate"]
                                  },
                            "title": document_name.replace(".pdf",""),
                            "id":id_name,
                            "wordingTitle":wording_name.replace(".pdf",""),
                            "matchType": total_category_matchType,

                            "relevance":total_category_relevance,

                            "refCode": document["refCode"],

                             "country":"GB",
                            "lob":document["lob"],
                            "matches": matches
                            }
                f=open("bb"+document['title']+".json","w")

                f.write(json.dumps(insert_item))
                print("*************************************************")  
                try:
                    trace_result_container.create_item(body=insert_item)
                except Exception as e:
                    print("exception block",wording_name,document_name,e)
                    try:
                        trace_result_container.replace_item(item=document_name, body=insert_item)
                    except Exception as e1:
                        print("exception block1",e1,wording_name,document_name,e1)
        except Exception as err:
            print("higher level exception",err,wording_name,document_name)

Chubb Multinational PDBI - UK - 04-2018 (SE - from 01-01-2019) - SPECIMENREDACTED_FB1D7BDB-BD05-4275-BE2A-DB30544D967F
Already Exist........


In [63]:
# new_keys={}
# for key_w in word_exclusion.keys():
#     keyFlag=False
#     key_wording=key_w
#     if key_w not in doc_exclusion.keys():
#         for key_doc in doc_exclusion.keys():

#             if key_w in key_doc or key_doc in key_w:
#                 key_wording=key_w
#                 key_document=key_doc
#                 keyFlag=True
#                 print(key_wording,"*",key_document)
#     else:
#         key_wording=key_w
#         key_document=key_w
#         print(key_wording,"*",key_document)
#         keyFlag=True
        
#     if not keyFlag:
#         print(key_w)
        
    
    
                    
            
     

In [64]:
document_name

'REDACTED_FB1D7BDB-BD05-4275-BE2A-DB30544D967F.pdf'

In [61]:
dist_

[(' ',
  '1959$b) overnight such vehicle be housed in a securely locked building or compound;'),
 (' ',
  '2440$ i subsidence, ground heave or landslip: i. to any Building (or Contents therein) arising from the settlement or movement of made-up ground or of ground over mine workings or on any site where there has been a previous occurrence of subsidence, ground heave or landslip; ii. to land, yards, car-parks, roads, pavements, walls, gates or fences unless a Building insured hereby is also affected; iii. resulting from coastal or river erosion or from any building, demolition or excavation works at the Premises unless resulting from fire, explosion, Earthquake or the escape of water from any tank, apparatus or pipe, or iv. commencing prior to the granting of this insurance; j its own collapse or the normal settling, cracking, shrinkage or expansion of the building or the site or any parts thereof on which the property described within stands; k the undernoted perils occurring while an

In [ ]:
a={"Conditions":"abc"}
b={"special Conditions":"def"}

In [ ]:

from itertools import product
from fuzzywuzzy import fuzz


partial_ratio = fuzz.partial_ratio( "special Conditions"," Section 4 Conditions")
print(partial_ratio)

In [ ]:
#                         if total_sections!=0:
#                             total_section_score= round(total_section_score/total_sections,2)

#                         total_section_matchType="Full"
#                         if total_section_score<99.5:
#                             total_section_matchType="partial"